# 3. Calling Twitter API

## 3.1 store twitter credential into Json File

In [51]:
import tweepy
import csv
import json
from tweepy.streaming import StreamListener
from tweepy import Stream

## 3.2 Get tweets from Twitter 

In [ ]:
# Twitter API credentials

with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_TOKEN']
    access_secret = info['ACCESS_SECRET']
    
# Create the api endpoint

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

myfile = open("twitter_data_kill.txt", 'w+')

In [ ]:
class listener(StreamListener):
    
    count = 0
    
    def on_data(self, data):
        all_data = json.loads(data)
        
        tweet = all_data["text"]
        place_countryCode = 'None'
        place_fullName = 'None'
        place_coordinates = 'None'
        place = all_data['place']
        if all_data['place'] != None:
            place_countryCode = all_data['place']['country_code']
            place_fullName = all_data['place']['full_name']
            place_coordinates = json.dumps(all_data['place']['bounding_box']['coordinates'])    
        if 'extended_tweet' in all_data:
            tweet = all_data['extended_tweet']['full_text']
        username = all_data["user"]["screen_name"]
        user_location = all_data["user"]["location"]
        id = all_data["user"]["id"]
        geo_enabled = all_data["user"]["geo_enabled"]
        tweet = tweet.replace('\n', ' ').replace('\r', '')
        myfile.write(str(id) + '|' + str(username) + '|' + str(user_location) + '|' + str(tweet) + '|' + str(geo_enabled) + '|' + str(place_countryCode) + '|' + str(place_fullName) + '|' + str(place_coordinates) + "\n")        
        print((id,username,user_location,tweet,geo_enabled,place_countryCode, place_fullName, place_coordinates))
        self.count = self.count + 1
        print(self.count)
        return True

    def on_error(self, status):
        print(status)

In [ ]:
auth.set_access_token(access_key, access_secret)
twitterStream = Stream(auth, listener())
LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,        # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,         # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]        # Hawaii
twitterStream.filter(track=['suicidal', 'suicide', 'kill myself', 'my suicide note'
                            'my suicide', 'letter', 'end my life', "never wake up", "can't go on", 'not worth living'
                            'ready to jump', 'sleep forever', 'want to die', 'be dead', 'better off without me', 
                            'better off dead', 'suicide plan', 'suicide pact', "tired of living", "don't want to be here", 
                            'die alone', "go to sleep forever"], locations=LOCATIONS)

## 3.3 Import twitter data 

In [179]:
import pandas as pd
import nltk
from textblob import Word 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

test = pd.read_csv("/Users/yeezhianliew/Desktop/twitter_data_us_new_clean.txt") 

In [180]:
test.head()

,Unnamed: 0,User_id,Username,User_Location,Tweet,Geo_Enabled,Place_CountryCode,Place_FullName,Place_Coordinates,centroid,latitude,longitude,Text,Retweet,P_POS,P_NEG,sentiment
0,0,412770174,momariedavison,"Minnetonka, MN",truly sooo glad i’ve been distancing myself fr...,True,US,"Prior Lake, MN","[[[-93.489043, 44.685054], [-93.489043, 44.761...","(44.723144500000004, -93.434323)",44.723144,-93.434323,truly sooo glad i’ve been distancing myself fr...,False,0.766,0.00,1
1,1,74008370,juarus,"Gadsden, Alabama",little smoked chicken potpie action eastsidepi...,True,US,"Gadsden, AL","[[[-86.079043, 33.933091], [-86.079043, 34.073...","(34.003125, -86.00682950000001)",34.003125,-86.006830,A little smoked chicken potpie action#eastside...,False,0.000,0.00,0
2,2,844713218,Tessv234,"San Marcos, Texas",u had to show hem the lizzie mcguire episode o...,True,US,"San Marcos, TX","[[[-97.993537, 29.840708], [-97.993537, 29.950...","(29.8956065, -97.9351915)",29.895606,-97.935192,@fleetw0odsnacc U had to show hem the Lizzie M...,False,0.218,0.00,1
3,3,321221241,jpenntaughtme,"No Name, CO",clown now we know everything you say is just ...,True,US,"Colorado, USA","[[[-109.060257, 36.992427], [-109.060257, 41.0...","(38.997935999999996, -105.5508905)",38.997936,-105.550891,"You been a clown, now we know everything you s...",False,0.000,0.38,-1
4,4,926663250036514816,rosegirl14hotm2,United States,love it here comes the tide rolling,True,US,"Alabama, USA","[[[-88.473228, 30.144425], [-88.473228, 35.008...","(32.576227, -86.68073749999999)",32.576227,-86.680737,"@AlabamaDieHards I love it, here comes the Ti...",False,0.824,0.00,1


In [181]:
test['Tweet']=test['Tweet'].fillna("")  

In [182]:
import nltk
from textblob import Word 
test['lower_case']= test['Tweet'].apply(lambda x: x.lower())       #convert upper to lower case

tokenizer = RegexpTokenizer(r'\w+')
test['Special_word'] = test.apply(lambda row: tokenizer.tokenize(row['lower_case']), axis=1)     #tokenize word

freq = pd.Series(' '.join(test['Tweet']).split()).value_counts()[-10:]                       
freq = list(freq.index)
test['Contents'] = test['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))  #remove less frequent words

stop = stopwords.words('english')
test['stop_words'] = test['Special_word'].apply(lambda x: [item for item in x if item not in stop])   #remove stop word

test['stop_words'] = test['stop_words'].astype('str')
test['short_word'] = test['stop_words'].str.findall('\w{3,}')            #remove words less than 3 characters
test['string'] = test['stop_words'].replace({"'": '', ',': ''}, regex=True)
test['string'] = test['string'].str.findall('\w{3,}').str.join(' ') 

nltk.download('words')
words = set(nltk.corpus.words.words())
test['NonEnglish'] = test['string'].apply(lambda x: " ".join(x for x in x.split() if x in words))  #remove non english word

test['tweet'] = test['NonEnglish'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) # convert it into root words
test.head()

[nltk_data] Downloading package words to
[nltk_data]     /Users/yeezhianliew/nltk_data...
[nltk_data]   Package words is already up-to-date!


,Unnamed: 0,User_id,Username,User_Location,Tweet,Geo_Enabled,Place_CountryCode,Place_FullName,Place_Coordinates,centroid,...,P_NEG,sentiment,lower_case,Special_word,Contents,stop_words,short_word,string,NonEnglish,tweet
0,0,412770174,momariedavison,"Minnetonka, MN",truly sooo glad i’ve been distancing myself fr...,True,US,"Prior Lake, MN","[[[-93.489043, 44.685054], [-93.489043, 44.761...","(44.723144500000004, -93.434323)",...,0.00,1,truly sooo glad i’ve been distancing myself fr...,"[truly, sooo, glad, i, ve, been, distancing, m...",truly sooo glad i’ve been distancing myself fr...,"['truly', 'sooo', 'glad', 'distancing', 'speci...","[truly, sooo, glad, distancing, specific, peop...",truly sooo glad distancing specific people fee...,truly glad specific people feel becoming happi...,truly glad specific people feel becoming happi...
1,1,74008370,juarus,"Gadsden, Alabama",little smoked chicken potpie action eastsidepi...,True,US,"Gadsden, AL","[[[-86.079043, 33.933091], [-86.079043, 34.073...","(34.003125, -86.00682950000001)",...,0.00,0,little smoked chicken potpie action eastsidepi...,"[little, smoked, chicken, potpie, action, east...",little smoked chicken potpie action eastsidepi...,"['little', 'smoked', 'chicken', 'potpie', 'act...","[little, smoked, chicken, potpie, action, east...",little smoked chicken potpie action eastsidepi...,little smoked chicken potpie action,little smoked chicken potpie action
2,2,844713218,Tessv234,"San Marcos, Texas",u had to show hem the lizzie mcguire episode o...,True,US,"San Marcos, TX","[[[-97.993537, 29.840708], [-97.993537, 29.950...","(29.8956065, -97.9351915)",...,0.00,1,u had to show hem the lizzie mcguire episode o...,"[u, had, to, show, hem, the, lizzie, mcguire, ...",u had to show hem the lizzie mcguire episode o...,"['u', 'show', 'hem', 'lizzie', 'mcguire', 'epi...","[show, hem, lizzie, mcguire, episode, singing,...",show hem lizzie mcguire episode singing christ...,show hem episode singing special suite life,show hem episode singing special suite life
3,3,321221241,jpenntaughtme,"No Name, CO",clown now we know everything you say is just ...,True,US,"Colorado, USA","[[[-109.060257, 36.992427], [-109.060257, 41.0...","(38.997935999999996, -105.5508905)",...,0.38,-1,clown now we know everything you say is just ...,"[clown, now, we, know, everything, you, say, i...",clown now we know everything you say is just s...,"['clown', 'know', 'everything', 'say', 'dumb',...","[clown, know, everything, say, dumb, shit, get...",clown know everything say dumb shit get people...,clown know everything say dumb get people talk...,clown know everything say dumb get people talk...
4,4,926663250036514816,rosegirl14hotm2,United States,love it here comes the tide rolling,True,US,"Alabama, USA","[[[-88.473228, 30.144425], [-88.473228, 35.008...","(32.576227, -86.68073749999999)",...,0.00,1,love it here comes the tide rolling,"[love, it, here, comes, the, tide, rolling]",love it here comes the tide rolling,"['love', 'comes', 'tide', 'rolling']","[love, comes, tide, rolling]",love comes tide rolling,love comes tide rolling,love come tide rolling


## 3.4 Tokenized

In [183]:
n_most_common_words =5000
max_len = 30
tokenizer = Tokenizer(num_words=n_most_common_words)
tokenizer.fit_on_texts(test['tweet'].values)
seq = tokenizer.texts_to_sequences(test['tweet'].values)
padded = pad_sequences(seq, maxlen=max_len)

## 3.5 Load Sentiment model

In [184]:
json_file = open('/Users/yeezhianliew/Desktop/model1.json', 'r')                  
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)                        #load the model
loaded_model.load_weights("/Users/yeezhianliew/Desktop/model1.h1")       #load weights into new model
print("Loaded model from disk")

Loaded model from disk


##  3.6 Predict Sentiment on Twitter data

In [175]:
result = loaded_model.predict(padded)
print(result)

[[0.28025565 0.7197443 ]
 [0.31371388 0.68628615]
 [0.79426694 0.20573306]
 ...
 [0.45268574 0.54731417]
 [0.45268577 0.5473142 ]
 [0.45268577 0.5473142 ]]


In [185]:
testing1 = pd.DataFrame(result)
testing1.columns = ["Positive", "Negative"]
testing1['Sentiment'] = testing1.idxmax(axis=1)
testing1.head()

,Positive,Negative,Sentiment
0,0.280256,0.719744,Negative
1,0.313714,0.686286,Negative
2,0.794267,0.205733,Positive
3,0.293821,0.706179,Negative
4,0.424152,0.575848,Negative


In [187]:
n_most_common_words =1000
max_len = 20
tokenizer = Tokenizer(num_words=n_most_common_words)
tokenizer.fit_on_texts(test['tweet'].values)
seq = tokenizer.texts_to_sequences(test['tweet'].values)
padded = pad_sequences(seq, maxlen=max_len)

## 3.7 Load Suicide Ideation model

In [188]:
json_file = open('/Users/yeezhianliew/Desktop/model_w.json', 'r')                  
loaded_model_json = json_file.read()
json_file.close()
loaded_models = model_from_json(loaded_model_json)                      #load the model
loaded_models.load_weights("/Users/yeezhianliew/Desktop/model_w.h5")    #load weights into new model
print("Loaded model from disk")

Loaded model from disk


##  3.8 Predict Suicide Ideation on Twitter data

In [189]:
results = loaded_models.predict(padded)
print(results)

[[0.03479315 0.96520686]
 [0.02105558 0.9789444 ]
 [0.04934056 0.9506594 ]
 ...
 [0.01289478 0.98710525]
 [0.01289478 0.98710525]
 [0.02849706 0.9715029 ]]


In [190]:
testing2 = pd.DataFrame(results)                                          # Create a Data frame for result
testing2.columns = ["Potential_Suicide_Post", "Not_Suicide_Post"]
testing2['Suicide_Ideation'] = testing2.idxmax(axis=1)
testing2.head()

,Potential_Suicide_Post,Not_Suicide_Post,Suicide_Ideation
0,0.034793,0.965207,Not_Suicide_Post
1,0.021056,0.978944,Not_Suicide_Post
2,0.049341,0.950659,Not_Suicide_Post
3,0.095111,0.904889,Not_Suicide_Post
4,0.017683,0.982317,Not_Suicide_Post


In [191]:
final_result = pd.DataFrame({'User_id':test['User_id'],'Username':test['Username'],
                             'User_location':test['User_Location'],'Place_CountryCode' :test['Place_CountryCode'],
                             'Place_FullName':test['Place_FullName'],'Original_Tweet':test['Tweet'],'Clean_Tweet':test['tweet'],  
                             'Place_Coordinates':test['Place_Coordinates'],'Sentiment':testing1['Sentiment'],
                             'Suicide_Ideation':testing2['Suicide_Ideation']})    #create the full result csv file
final_result.to_csv('/Users/yeezhianliew/Desktop/output.csv',index=False)